# BeautifulSoup

In [1]:
import numpy as np # pandas 라이브러리 의존성 때문에
import pandas as pd # 가져온 데이터를 가공하기 위해
from tqdm import tqdm_notebook # 진행률을 표시하기 위해
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
url_base = 'https://finance.naver.com/marketindex/exchangeList.nhn' #요청할 URL

page = urlopen(url_base) #요청해서 가져온 내용

soup = BeautifulSoup(page,'html.parser') #page를 html.parser로 parse해 저장

In [3]:
#print(soup) #크롬-개발자 도구 코드와 동일

## select 함수
- soup.select("title")
- soup.select("p:nth-of-type(3)")
- soup.select("body a")
- soup.select("html head title")
- soup.select("head > title")
- soup.select("p > a")
- soup.select(".sister")
- soup.select("[class~=sister]")
- soup.select("#link1")
- soup.select("a#link2")

In [4]:
cur_name = [td.a.string.strip() for td in soup.select('td.tit')] #td중 class가 tit인 요소
cur_name[:5]

['미국 USD', '유럽연합 EUR', '일본 JPY (100엔)', '중국 CNY', '홍콩 HKD']

In [5]:
sale = [ td.string.strip() for td in soup.select('td.sale')]
sale[:5]

['1,218.00', '1,359.35', '1,146.51', '173.94', '156.78']

## find 함수
find는 하나만 반환하기 때문에 굳이 find를 쓰기 보다 find_all을 쓰는 것이 좋음

- soup.find_all("title")
- soup.find_all("p", "title")
- soup.find_all("a")
- soup.find_all(id="link2")
- import re
- soup.find(string=re.compile("sisters"))

In [6]:
trs = soup.find('tbody').find_all('tr') # tbody 태그를 찾고 재귀적으로 다시 tr 태그를 찾음
#트리 구조로 찾을 수 있음

trs[0] # 반환형 리스트이므로 첫번재 원소를 확인

<tr>
<td class="tit"><a href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" onclick="parent.clickcr(this, 'exl.exlist', 'FX_USDKRW', '1', event);" target="_parent">
				
					
					
					
					미국 USD
				
				</a></td>
<td class="sale">1,218.00</td>
<td>1,239.31</td>
<td>1,196.69</td>
<td>1,229.90</td>
<td>1,206.10</td>
<td>1.000</td>
</tr>

In [7]:
trs[0].find_all('td')[0].string.strip()

'미국 USD'

## Table 만들기
pandas DataFrame 형태로 저장

In [8]:
item = list()
for i in range(0,len(trs)):
    tds = trs[i].find_all('td')
    item.append([tds[j].string.strip() for j in range(0,len(tds))])

In [9]:
table = pd.DataFrame(item)
# 통화명, 매매기준율, 현찰-살때, 현찰-팔때, 송금-보낼때, 송금-받을떄, 미화환산율
table.columns = ('curname','sale','buy','sell','remit','deposit','us_ex')
table.head()

,curname,sale,buy,sell,remit,deposit,us_ex
0,미국 USD,"1,218.00","1,239.31","1,196.69","1,229.90","1,206.10",1.000
1,유럽연합 EUR,"1,359.35","1,386.40","1,332.30","1,372.94","1,345.76",1.116
2,일본 JPY (100엔),"1,146.51","1,166.57","1,126.45","1,157.74","1,135.28",0.941
3,중국 CNY,173.94,182.63,165.25,175.67,172.21,0.143
4,홍콩 HKD,156.78,159.86,153.70,158.34,155.22,0.129


## csv로 저장
to_csv 함수로 DB 저장

# Selenium

In [11]:
import numpy as np # pandas 라이브러리 의존성 때문에
import pandas as pd # 가져온 데이터를 가공하기 위해
from tqdm import tqdm_notebook # 진행률을 표시하기 위해
from selenium import webdriver as wd # Chrome 웹드라이버 준비
import time # selenium은 sleep을 걸어야해서
import urllib
import platform # https://pinkwink.kr/1002
import getpass # https://pwnbit.kr/22

In [ ]:
# Selenium 드라이버
# 운영체제에 맞게 드라이버 선택
current_os = platform.system()
if current_os == 'Windows': # windows 사용자일 경우
    driver = wd.Chrome('./tool/Windows/chromedriver')
elif current_os == 'Darwin': # macOS 사용자일 경우
    driver = wd.Chrome('./tool/macOS/chromedriver')
  
#패스 수정
#driver = webdriver.Chrome('/path/to/chromedriver') 

## youtube 로그인하기

In [ ]:
# YouTube 접속하기
driver.get('https://www.youtube.com/')

In [ ]:
# YouTube 로그인하는 부분
# macOS와 Windows와 css 선택자를 다르게 선택해야함..
login_button = driver.find_element_by_css_selector('.style-scope.ytd-button-renderer.style-destructive.size-default')
login_button.click()

In [ ]:
# 로그인할 이메일
email = '' # 본인의 이메일을 입력하자
driver.find_element_by_id('identifierId').send_keys(email)
driver.find_element_by_css_selector('.CwaK9').click()

In [ ]:
# 로그인할 비밀번호
password = getpass.getpass()
driver.find_element_by_css_selector('.whsOnd.zHQkBf').send_keys(password)
driver.find_element_by_id('passwordNext').click()

## youtube 영상 제목 수집하기
- 브라우저 조작 함수 : execute_script
- 정보 탐색 함수
    - find_elements_by_id
    - find_elements_by_tag_name
    - find_element_by_css_selector

In [ ]:
# 접속할 사이트
# base_url = 'https://www.youtube.com/results'
# params = '?search_query=%s'
# keyword = urllib.parse.quote('BOAZ 빅데이터')
# target_url = base_url + (params%keyword)
# 접속
# driver.get(target_url)

keyword = 'BOAZ 빅데이터'
driver.find_element_by_id('search').send_keys(keyword)
driver.find_element_by_id('search-icon-legacy').click()

In [ ]:
# 스크롤 이벤트로 정보를 더 load할 수 있음
for n in range(5):
    driver.execute_script('window.scrollBy(0,1000)') # 자바스크립트를 돌리는 코드
    time.sleep(1) # selenium에서는 sleep을 걸어주지 않으면 bot으로 인식당할 수 있음

In [ ]:
# Youtube에 검색된 BOAZ 관련 영상들
videos = driver.find_elements_by_tag_name('ytd-video-renderer') # ytd-playlist-renderer안에는 좋아요가 없으므로
print('%d개의 영상을 찾았습니다!!'%len(videos))

In [ ]:
boaz_videos = list()
boaz_playlist = list()
for v in videos:
    # BOAZ와 무관한 영상들에 관해서 처리
    # 영상제목에 BOAZ나 boaz가 있을시 해당
    video_title = v.find_element_by_id('video-title').text
    print(video_title)
    if (('BOAZ' in video_title) or ('boaz' in video_title) or ('보아즈' in video_title)):
        boaz_videos.append(v)

In [ ]:
for v in boaz_videos:
    boaz_video_title = v.find_element_by_id('video-title').text
    boaz_video_url = v.find_element_by_id('video-title').get_attribute('href')
    boaz_playlist.append([boaz_video_title,boaz_video_url])

In [ ]:
df = pd.DataFrame(boaz_playlist,columns=('Title','URL'))
df

## 좋아요 버튼 누르기

In [ ]:
for i in range(20,25):
    print('%2d번째 BOAZ 영상 작업중'%(i+1), end=' ')
    driver.get(boaz_playlist[i][1])
    driver.implicitly_wait(10) # 충분한 시간을 주지 않을 경우, load에 실패할 가능성이 높아진다
    liked = driver.find_elements_by_css_selector('.style-scope.ytd-toggle-button-renderer.style-default-active')
    # print(len(liked))
    if not liked: # 좋아요의 css가 없을 경우, ( 즉, 좋아요 눌러야할 영상일 경우 )
        buttons = driver.find_elements_by_css_selector('.style-scope.ytd-toggle-button-renderer.style-text')
        buttons[0].click() # 1번째로 찾은 버튼이 좋아요 버튼이라서
        print('-> 좋아요 누름!')
        continue
    print('-> 이미 눌려있음!')

# API
https://developers.naver.com/main/ 에서 미리 오픈 API 신청 후 사용

In [ ]:
import requests
import getpass

In [ ]:
client_key = '' # 미리 API키를 신청받아야함
client_secret = getpass.getpass() 

## 네이버 뉴스 검색

In [ ]:
keyword = 'BOAZ,빅데이터'

In [ ]:
# 별도 quote_plus() 메서드등 처리할 필요 없음. requests 객체가 알아서 해줌
naver_url = 'https://openapi.naver.com/v1/search/news.json?query=%s'%keyword

# 요청을 할때, 네이버에서 API를 발급받은 사람인지 확인하기 위해서
# Client ID 와 Client Secret를 요구하기 때문에 같이 보내줘야한다
header_params = {"X-Naver-Client-Id":client_key,
                 "X-Naver-Client-Secret":client_secret}

In [ ]:
# headers= header_params 는 header 변경시에만 필요하고, 그렇지 않으면, requests.get(원하는 URL) 만 해도 됨
response = requests.get(naver_url, headers = header_params)

# 별도 json.loads() 라이브러리 메서드 사용하지 않아도, reqeusts 라이브러리에 있는 json() 메서드로 간단히 처리 가능함
print(response.json())
# print(response.text)

In [ ]:
# HTTP 응답 코드는 status_code 에 저장됨
# 성공할 시, 코드 200을 의미함
if(response.status_code == 200):
    data = response.json()
    print(data['items'][0]['title'])
    print(data['items'][0]['description'])
else:
    print("Error Code:" + str(response.status_code))

# 이미지 크롤링 예제

## BeautifulSoup 이용

In [ ]:
from urllib.request import urlopen # 내장함수
import requests as req # request => 요청하는거를 웹에 요청한 결과값을 얻어올수 있는 모듈
from bs4 import BeautifulSoup # 웹에 요청한 결과를 보내주는 모듈
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm_notebook
import getpass
import urllib

In [ ]:
def get_by_bs4(keyword):

    # 사용한 구글 url https://www.google.co.kr/search?q=%EB%B2%A4&tbm=isch
    url_info = "https://www.google.co.kr/search?"

    #params에 딕션을 넣어줌
    params = {
        #명령행에서 받은 인자값을 people로 넣어줌
        "q" : keyword, #이미지 크롤링을 위해 항상 이 값 유지
        "tbm":"isch" #이미지 크롤링을 위해 항상 이 값 유지
    }
    #url 요청 파싱값
    html_object = req.get(url_info,params) #html_object html source 값

    #페이지 status_code 가 200 일때 2XX 는 성공을 이야기함
    if html_object.status_code == 200:
        
        #인스턴스 생성
        bs_object = BeautifulSoup(html_object.text,"html.parser")
        
        #인스턴스의 find_all 이라는 함수에 img 태그가 있으면 img_data에 넣어줌
        img_data = bs_object.find_all("img")
        
        # 이미지의 갯수를 세기 위한 객체
        cnt = 1
        
        for i in enumerate(img_data[1:5]):
            #딕셔너리를 순서대로 넣어줌
            t = urlopen(i[1].attrs['src']).read()

            filename = './img/bs4/'+str(keyword)+'-'+str(i[0]+1)+'.jpg'

            with open(filename,"wb") as f:
                f.write(t)
            print("%d번째 %s 이미지 저장완료"%(cnt,keyword))
            cnt += 1
            
            img = mpimg.imread(filename)
            imgplot = plt.imshow(img)
            plt.show()

In [ ]:
get_by_bs4('빅데이터 BOAZ')

## API 이용

In [24]:
client_id = '' # API 신청해서 발급받은 ID
client_secret = getpass.getpass() # 2QJ5N39nnB

········


In [ ]:
def make_naver_search_api_url(node, search_text, start_num, disp_num):
    base_url = 'https://openapi.naver.com/v1/search/' + node + '.json'
    param_query = "?query=" + urllib.parse.quote(search_text)
    param_start = "&start=" + str(start_num)
    param_disp = "&display=" + str(disp_num)
    
    return base_url + param_query + param_start + param_disp

In [ ]:
def get_request_url(API_url, client_id, client_secret):
    import json
    request = urllib.request.Request(API_url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    request.add_header("Content-Type","application/json")
    
    response = urllib.request.urlopen(request)
    if response.getcode() == 200:
        return json.load(response)
    else:
        return None
        print("--- error ---")

In [ ]:
keyword = '빅데이터 BOAZ'
api_url = make_naver_search_api_url('image', keyword, 1, 10)
result = get_request_url(api_url, client_id, client_secret)

i = 0
items = result['items']
for d in tqdm_notebook(items):
    filename = str(keyword)+'-'+str(i)
    savepath = './img/api/'
    urllib.request.urlretrieve(items[i]['thumbnail'],savepath+filename+'.jpg')
    print(items[i]['link'][:80] + ' 이미지 저장')
    i+=1

## 그 외 크롤러

In [ ]:
from icrawler.builtin import GoogleImageCrawler

google_crawler = GoogleImageCrawler(storage={'root_dir': './img/etc'})
google_crawler.crawl(keyword='cat', max_num=10)